<a href="https://colab.research.google.com/github/SLy-Precision/Player-Detection-and-Team-Performance-Prediction/blob/main/04_AEGIS_Simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 🔮 AEGIS Simulations 🧪 🎰

## ⚽ Once we did the scouting using advanced metrics (KPI's) now its time to fine-tunning teams with the new player in roster, here we are gonna get the proyections of performance of Club America with these additions 🕹️

In [4]:
# Lets import the libraries we are gonna use 🐍

# Statsbomb api and more
!pip install --quiet statsbombpy
!pip install --quiet mplsoccer
!pip install --quiet highlight_text
!pip install --quiet xgboost

# Libraries for the data manipulation and plot
from statsbombpy import sb
from tqdm import tqdm
import pandas as pd
from mplsoccer import VerticalPitch, Pitch, Sbapi
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
import numpy as np
from collections import defaultdict
from collections import Counter
from itertools import groupby
from mplsoccer import Pitch
import random

# Machine Learning

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
import matplotlib.pyplot as plt

In [5]:
# Access to drive and files

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Proyect_Hackathon/'

# Data needed

df_AEGIS_simulation = pd.read_pickle(path + 'df_with_context0.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
id_america = 1229
team_name_prefix = 'club_america'

### 🤼‍♀ Analysis of Substitutions

In [ ]:
array_recomendations = []
array_replaced = []

In [ ]:
# Lets see how many events had every player agains his replacement in tournament and verify coherence in substitutions

## ⚠️ Simulation Player 1: